In [1]:
from transformers import RobertaTokenizerFast, AutoModelForTokenClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, log_loss, accuracy_score, matthews_corrcoef
from sklearn.metrics import roc_auc_score
from scipy.special import softmax
import pandas as pd
import pickle
import numpy as np
import torch
import pickle5 as pickle

/home/johnmf4/.conda/envs/ProteinTransformers3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
tokenizer = RobertaTokenizerFast.from_pretrained("./Models/ST-PRoBERTa/Tokenizer")
model = AutoModelForTokenClassification.from_pretrained("./Models/DR-BERT-Frozen-Embedding/Checkpoints/checkpoint-15690")

file ./Models/ST-PRoBERTa/Tokenizer/config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
file ./Models/ST-PRoBERTa/Tokenizer/config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
device = torch.device('cuda')
model.to(device)

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(35, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

In [5]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

43345922

In [6]:
def model_out(encoded, model):
    encoded.to(device)
    with torch.no_grad():
        output = model(**encoded)
    return output

In [7]:
def get_word_vector(sent, tokenizer, model):
    encoded = tokenizer.encode_plus(sent, return_tensors="pt", truncation=True, max_length=1024)
    out = model_out(encoded, model)
    return out

In [8]:
# import pickle5 as pickle
import pickle

In [9]:
df_caid = pd.read_pickle('./Datasets/caid.pkl')
df_caid_winners_test = pd.read_pickle('./Datasets/caid_winners_test.pkl')
df_casp = pd.read_pickle('./Datasets/casp.pkl')
df_test = pd.read_pickle('../DR-BERT/Datasets/finetuning-IDRs-test.pickle')
df_train = pd.read_pickle('../DR-BERT/Datasets/finetuning-IDRs-train.pickle')
df_full = pd.concat((df_test, df_train))
df_fdi = pd.DataFrame(pd.read_pickle(open('./Datasets/fIDPnnClass_test (2).pkl', 'rb')), columns=['ID', 'Sequence', 'labs', 'Protein', 'DNA', 'RNA', 'Linker'])
df_caid_protein = pd.DataFrame(pd.read_pickle(open('./Datasets/caid_protein2.pkl', 'rb')), columns=['ID', 'Sequence', 'labs'])

In [11]:
Xs = np.concatenate(Xs)

In [12]:
print(Xs.shape)

(298040, 2)


In [16]:
def precision_recall_f1_roc_convolve(name, logits, labels, convolution):
    logits = [i[1] for i in logits]
    convolved = np.convolve(np.array(logits).flatten(), np.array(convolution / np.sum(convolution).flatten()), 'same')
    p = [(1 - i, i) for i in convolved]
    roc = [i[1] for i in p]
    print(len(p))
    p = np.argmax(p, axis=-1)
    precision, recall, f1, support = precision_recall_fscore_support(labels, p)
    roc_auc = roc_auc_score(labels, roc)
    mcc = matthews_corrcoef(labels, p)
    return {
        f'precision_{name}':precision[1],
        f'recall_{name}':recall[1],
        f'f1_{name}':f1[1],
        f'roc_auc_{name}':roc_auc,
        f'MCC_{name}':mcc,
    }

In [10]:
def compute_metrics(logits, labels):
    logits = softmax(logits, axis=1)
    l = []
    for j, i in enumerate(labels):
        l = l + list(i)
    print(len(l), len(logits))
    lg2 = logits
    metrics = {}
    metrics.update(precision_recall_f1_roc_convolve('normal', lg2, l, [1]))
    metrics.update(precision_recall_f1_roc_convolve('wa5', lg2, l, [1,1,1,1,1]))
    metrics.update(precision_recall_f1_roc_convolve('wa9', lg2, l, [1,1,1,1,1,1,1]))
    metrics.update(precision_recall_f1_roc_convolve('wa15', lg2, l, [1]*15))
    metrics.update(precision_recall_f1_roc_convolve('linear5', lg2, l, [1,2,3,2,1]))
    metrics.update(precision_recall_f1_roc_convolve('linear9', lg2, l, [1,2,3,4,5,4,3,2,1]))
    metrics.update(precision_recall_f1_roc_convolve('linear15', lg2, l, [1,2,3,4,5,6,7,8,7,6,5,4,3,2,1]))
    metrics.update(precision_recall_f1_roc_convolve('quad5', lg2, l, [1,3,9,3,1]))
    metrics.update(precision_recall_f1_roc_convolve('quad9', lg2, l, [1,3,9,27,81,27,9,3,1]))
    metrics.update(precision_recall_f1_roc_convolve('quad15', lg2, l, [1,3,9,27,81,243,729,2187,729,243,81,27,9,3,1]))
    
    return metrics

In [11]:
Xs = []
labs = []
for index, row in df_caid.iterrows():
    seq = row['Sequence']
    lab = list(row['full'])
#     lab = list(row['labs'])
    word_embedding = get_word_vector(seq[:min(1024, len(seq))], tokenizer, model)
    word_embedding = word_embedding['logits'][0]
    word_embedding = word_embedding[1:-1]
    word_embedding = word_embedding.cpu()
    word_embedding_full = np.squeeze(word_embedding.numpy())
    print(word_embedding.shape)
    if len(seq) > 2044:
        for i in range(1022, len(seq) - 1022, 1022):
            word_embedding = get_word_vector(seq[i:i+1022], tokenizer, model)
            word_embedding = word_embedding['logits'][0]
            word_embedding = word_embedding[1:-1]
            word_embedding = word_embedding.cpu()
            word_embedding = np.squeeze(word_embedding.numpy())
            word_embedding_full = np.concatenate([word_embedding_full, word_embedding], axis=0)
    if len(seq) > 1024:
        word_embedding = get_word_vector(seq[len(seq) - 1022:len(seq)], tokenizer, model)
        word_embedding = word_embedding['logits'][0]
        word_embedding = word_embedding[1:-1]
        word_embedding = word_embedding.cpu()
        word_embedding = np.squeeze(word_embedding.numpy())
        overlap_amount = 1022 - (len(seq) % 1022)
        overlap = word_embedding_full[(-1 * overlap_amount):]
        word_embedding_full[(-1 * overlap_amount):] = (overlap + word_embedding[:overlap_amount]) / 2
        end_chunk = word_embedding[overlap_amount:]
        word_embedding_full = np.concatenate([word_embedding_full, end_chunk], axis=0)
    print(word_embedding_full.shape, len(seq))
    Xs.append(word_embedding_full)
    labs.append(lab[:len(seq)])

torch.Size([160, 2])
(160, 2) 160
torch.Size([362, 2])
(362, 2) 362
torch.Size([157, 2])
(157, 2) 157
torch.Size([491, 2])
(491, 2) 491
torch.Size([107, 2])
(107, 2) 107
torch.Size([196, 2])
(196, 2) 196
torch.Size([414, 2])
(414, 2) 414
torch.Size([353, 2])
(353, 2) 353
torch.Size([624, 2])
(624, 2) 624
torch.Size([440, 2])
(440, 2) 440
torch.Size([724, 2])
(724, 2) 724
torch.Size([708, 2])
(708, 2) 708
torch.Size([377, 2])
(377, 2) 377
torch.Size([605, 2])
(605, 2) 605
torch.Size([1022, 2])
(3969, 2) 3969
torch.Size([1022, 2])
(1050, 2) 1050
torch.Size([499, 2])
(499, 2) 499
torch.Size([781, 2])
(781, 2) 781
torch.Size([410, 2])
(410, 2) 410
torch.Size([243, 2])
(243, 2) 243
torch.Size([400, 2])
(400, 2) 400
torch.Size([619, 2])
(619, 2) 619
torch.Size([457, 2])
(457, 2) 457
torch.Size([262, 2])
(262, 2) 262
torch.Size([1022, 2])
(1430, 2) 1430
torch.Size([158, 2])
(158, 2) 158
torch.Size([651, 2])
(651, 2) 651
torch.Size([160, 2])
(160, 2) 160
torch.Size([655, 2])
(655, 2) 655
torch

torch.Size([229, 2])
(229, 2) 229
torch.Size([83, 2])
(83, 2) 83
torch.Size([798, 2])
(798, 2) 798
torch.Size([685, 2])
(685, 2) 685
torch.Size([807, 2])
(807, 2) 807
torch.Size([1022, 2])
(1482, 2) 1482
torch.Size([215, 2])
(215, 2) 215
torch.Size([1022, 2])
(1117, 2) 1117
torch.Size([655, 2])
(655, 2) 655
torch.Size([181, 2])
(181, 2) 181
torch.Size([956, 2])
(956, 2) 956
torch.Size([141, 2])
(141, 2) 141
torch.Size([224, 2])
(224, 2) 224
torch.Size([255, 2])
(255, 2) 255
torch.Size([274, 2])
(274, 2) 274
torch.Size([115, 2])
(115, 2) 115
torch.Size([290, 2])
(290, 2) 290
torch.Size([295, 2])
(295, 2) 295
torch.Size([191, 2])
(191, 2) 191
torch.Size([222, 2])
(222, 2) 222
torch.Size([676, 2])
(676, 2) 676
torch.Size([735, 2])
(735, 2) 735
torch.Size([71, 2])
(71, 2) 71
torch.Size([73, 2])
(73, 2) 73
torch.Size([53, 2])
(53, 2) 53
torch.Size([473, 2])
(473, 2) 473
torch.Size([259, 2])
(259, 2) 259
torch.Size([296, 2])
(296, 2) 296
torch.Size([282, 2])
(282, 2) 282
torch.Size([496, 2])

torch.Size([161, 2])
(161, 2) 161
torch.Size([555, 2])
(555, 2) 555
torch.Size([346, 2])
(346, 2) 346
torch.Size([374, 2])
(374, 2) 374
torch.Size([235, 2])
(235, 2) 235
torch.Size([346, 2])
(346, 2) 346
torch.Size([614, 2])
(614, 2) 614
torch.Size([394, 2])
(394, 2) 394
torch.Size([173, 2])
(173, 2) 173
torch.Size([150, 2])
(150, 2) 150
torch.Size([200, 2])
(200, 2) 200
torch.Size([86, 2])
(86, 2) 86
torch.Size([181, 2])
(181, 2) 181
torch.Size([172, 2])
(172, 2) 172
torch.Size([266, 2])
(266, 2) 266
torch.Size([436, 2])
(436, 2) 436
torch.Size([1022, 2])
(1113, 2) 1113
torch.Size([299, 2])
(299, 2) 299
torch.Size([49, 2])
(49, 2) 49
torch.Size([1022, 2])
(1739, 2) 1739
torch.Size([270, 2])
(270, 2) 270
torch.Size([532, 2])
(532, 2) 532
torch.Size([575, 2])
(575, 2) 575
torch.Size([713, 2])
(713, 2) 713
torch.Size([700, 2])
(700, 2) 700
torch.Size([1005, 2])
(1005, 2) 1005
torch.Size([964, 2])
(964, 2) 964
torch.Size([358, 2])
(358, 2) 358
torch.Size([274, 2])
(274, 2) 274
torch.Size(

In [13]:
Xs = np.concatenate(Xs)

In [14]:
Xs.shape

(338068, 2)

In [17]:
metrics = compute_metrics(Xs, labs)
print(metrics)

338068 338068
338068
338068
338068
338068


/home/johnmf4/.conda/envs/ProteinTransformers3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


338068
338068


/home/johnmf4/.conda/envs/ProteinTransformers3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


338068


/home/johnmf4/.conda/envs/ProteinTransformers3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


338068
338068
338068
{'precision_normal': 0.20212765957446807, 'recall_normal': 0.0003462225299755822, 'f1_normal': 0.0006912610056028525, 'roc_auc_normal': 0.5763590573831356, 'MCC_normal': 0.0017999666346789234, 'precision_wa5': 0.0, 'recall_wa5': 0.0, 'f1_wa5': 0.0, 'roc_auc_wa5': 0.5931005529703909, 'MCC_wa5': -0.0020031416684274553, 'precision_wa9': 0.0, 'recall_wa9': 0.0, 'f1_wa9': 0.0, 'roc_auc_wa9': 0.5949741544647473, 'MCC_wa9': -0.0007571096664040824, 'precision_wa15': 0.0, 'recall_wa15': 0.0, 'f1_wa15': 0.0, 'roc_auc_wa15': 0.5975230301997384, 'MCC_wa15': 0.0, 'precision_linear5': 0.0, 'recall_linear5': 0.0, 'f1_linear5': 0.0, 'roc_auc_linear5': 0.5921197704419814, 'MCC_linear5': -0.0025110858277663387, 'precision_linear9': 0.0, 'recall_linear9': 0.0, 'f1_linear9': 0.0, 'roc_auc_linear9': 0.5952340907159847, 'MCC_linear9': 0.0, 'precision_linear15': 0.0, 'recall_linear15': 0.0, 'f1_linear15': 0.0, 'roc_auc_linear15': 0.5970797013940405, 'MCC_linear15': 0.0, 'precision_quad5'

In [18]:
pickle.dump(Xs, open('Frozen_embeddings_logits.pickle', 'wb'))